# Classify snow-covered area (SCA) in Landsat, Sentinel-2, and MODIS surface reflectance imagery: full pipeline

Rainey Aberle

Department of Geosciences, Boise State University

2022

### Requirements:
- Area of Interest (AOI) shapefile: where snow will be classified in all available images. 
- Google Earth Engine (GEE) account: used to pull DEM over the AOI. Sign up for a free account [here](https://earthengine.google.com/new_signup/). 

### Outline:
__0. Setup__ paths in directory, AOI file location - _modify this section!_

__1. Load images__ over the AOI since 2016. 

__2. Prepare image collections__ for classification: reprojection and image quality masking. 

__3. Classify SCA__ and use the snow elevations distribution to estimate the seasonal snowline

-------


### 0. Setup

#### Define paths in directory and desired settings. 
Modify lines located within the following:

`#### MODIFY HERE ####`  

`#####################`

In [ ]:
##### MODIFY HERE #####
# -----Path to planet-snow
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping/'

# -----Paths in directory
site_name = 'Wolverine'
# path to AOI including the name of the shapefile
AOI_fn = base_path+'../../GIS_data/RGI_outlines/'+site_name+'_RGI.shp'
# path for output figures
figures_out_path = base_path+'../study-sites/'+site_name+'/figures/SCA_Landsat/'

# -----Define maximum cloud cover filter for image search
cloud_cover_max = 10

# -----Determine settings
plot_results = True # = True to plot figures of results for each image where applicable
crop_to_AOI = True # = True to crop images to AOI before calculating SCA
save_outputs = True # = True to save SCA images to file
save_figures = True # = True to save SCA output figures to file

#######################

# -----Import packages
import xarray as xr
import rioxarray
import wxee as wx
import os
import numpy as np
import glob
from osgeo import gdal
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib.patches import Rectangle
from matplotlib import pyplot as plt, dates
import rasterio as rio
import rasterio.features
from rasterio.mask import mask
from rasterio.plot import show
from shapely.geometry import Polygon, shape
import shapely.geometry
from scipy.interpolate import interp2d
from scipy import stats
import pandas as pd
import geopandas as gpd
import geemap
import math
import sys
import ee
import fiona
import pickle
import wxee as wx
import time

# -----Add path to functions
sys.path.insert(1, base_path+'functions/')
import ps_pipeline_utils as f

# -----Load AOI as geopandas.GeoDataFrame
AOI = gpd.read_file(AOI_fn)

#### Authenticate and initialize Google Earth Engine (GEE). 

__Note:__ The first time you run the following cell, you will be asked to authenticate your GEE account for use in this notebook. This will send you to an external web page, where you will walk through the GEE authentication workflow and copy an authentication code back in this notebook when prompted. 

In [ ]:
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

### 1. Load images over the AOI since 2016


In [ ]:
# -----Reformat AOI for clipping images
# reproject AOI to WGS 84 for compatibility with images
AOI_WGS = AOI.to_crs(4326)
# reformat AOI_WGS bounding box as ee.Geometry for clipping DEM
AOI_WGS_bb_ee = ee.Geometry.Polygon(
                        [[[AOI_WGS.geometry.bounds.minx[0], AOI_WGS.geometry.bounds.miny[0]],
                          [AOI_WGS.geometry.bounds.maxx[0], AOI_WGS.geometry.bounds.miny[0]],
                          [AOI_WGS.geometry.bounds.maxx[0], AOI_WGS.geometry.bounds.maxy[0]],
                          [AOI_WGS.geometry.bounds.minx[0], AOI_WGS.geometry.bounds.maxy[0]],
                          [AOI_WGS.geometry.bounds.minx[0], AOI_WGS.geometry.bounds.miny[0]]]
                        ])

# -----Define start and end dates and months
date_range_start = '2022-01-01'
date_range_end = '2022-12-01'
month_start = 5
month_end = 10

# -----Query GEE for Landsat, Sentinel, and MODIS imagery
L_col = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
         .filter(ee.Filter.lt("CLOUD_COVER", cloud_cover_max))
         .filterDate(ee.Date(date_range_start), ee.Date(date_range_end))
         .filter(ee.Filter.calendarRange(month_start, month_end, 'month'))
         .filterBounds(AOI_WGS_bb_ee))
S_col = (ee.ImageCollection("COPERNICUS/S2_SR")
         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover_max))
         .filterDate(ee.Date(date_range_start), ee.Date(date_range_end))
         .filter(ee.Filter.calendarRange(month_start, month_end, 'month'))
         .filterBounds(AOI_WGS_bb_ee))
M_col = (ee.ImageCollection("MODIS/061/MOD09GA").merge(ee.ImageCollection("MODIS/061/MYD09GA"))
         .filterDate(ee.Date(date_range_start), ee.Date(date_range_end))
         .filter(ee.Filter.calendarRange(month_start, month_end, 'month'))
         .filterBounds(AOI_WGS_bb_ee))

# -----Clip images to AOI and select bands
L_band_names = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
S_band_names = ['B2', 'B3', 'B4', 'B5', 'B6', 'B8', 'B11', 'B12']
M_band_names = ['sur_refl_'+x for x in ['b01', 'b04', 'b03', 'b05', 'b06', 'b07']]
def clip_image(im):
    return im.clip(AOI_WGS_bb_ee.buffer(2000))
L_col_clip = L_col.map(clip_image).select(L_band_names + ['QA_PIXEL'])
S_col_clip = S_col.map(clip_image).select(S_band_names + ['QA60'])
M_col_clip = M_col.map(clip_image).select(M_band_names + ['state_1km'])

# -----Convert image collections to xarray Datasets
print('Landsat image collection:')
L_col_xr = L_col_clip.wx.to_xarray(scale=30, crs='EPSG:4326')
print('Sentinel-2 image collection:')
S_col_xr = S_col_clip.wx.to_xarray(scale=20, crs='EPSG:4326')
print('MODIS image collection:')
M_col_xr = M_col_clip.wx.to_xarray(scale=500, crs='EPSG:4326')

### 2. Prepare image collections for classification: reproject images collections to UTM and apply quality masks

In [ ]:
# -----Reproject image collections to UTM
# Determine optimal UTM zone 
epsg_code = f.convert_wgs_to_utm((AOI_WGS.geometry.bounds.maxx[0] - AOI_WGS.geometry.bounds.minx[0]) + AOI_WGS.geometry.bounds.minx[0],
                                 (AOI_WGS.geometry.bounds.maxy[0] - AOI_WGS.geometry.bounds.miny[0]) + AOI_WGS.geometry.bounds.miny[0])
# Reproject using rasterio.reproject
L_col_xr_UTM = L_col_xr.rio.reproject("EPSG:"+epsg_code)
S_col_xr_UTM = S_col_xr.rio.reproject("EPSG:"+epsg_code)
M_col_xr_UTM = M_col_xr.rio.reproject("EPSG:"+epsg_code)

In [ ]:
# -----Apply image quality masking to Landsat and Sentinel imagery
# Landsat
# Pixel quality band = "QA_PIXEL"
# Bit 3 = cloud shadow, Bit 5 = cloud
# def L8_QA_mask(im):
#     cloudShadowBitMask = 1 << 3
#     cloudsBitMask = 1 << 5;
#     # Get the pixel QA band.
#     qa = im.select('QA_PIXEL')
#     # Both flags should be set to zero, indicating clear conditions.
#     mask = (qa.bitwiseAnd(cloudShadowBitMask).eq(0) & (qa.bitwiseAnd(cloudsBitMask).eq(0)))
#     # Return the masked image without the QA bands.
#     return (
#         image.updateMask(mask)
#         .select("SR_B[0-9]*")
#         .copyProperties(image, ["system:time_start"])
#     )

# L_col_clip_mask = L_col_clip.map(L8_QA_mask)


#   // Sentinel-2 image quality masking
#   function S2QAMask(image){
#     var QA60 = image.select(['QA60']);
#     return image.updateMask(QA60.lt(1));
#   }

### 3. Classify SCA

In [ ]:
# start timer
# t1 = time.time()

# -----Load image classifiers and feature columns
# Landsat
L_clf_fn = base_path+'inputs-outputs/L_classifier_all_sites.sav'
L_clf = pickle.load(open(L_clf_fn, 'rb'))
L_feature_cols_fn = base_path+'inputs-outputs/L_feature_cols.pkl'
L_feature_cols = pickle.load(open(L_feature_cols_fn,'rb'))
# Sentinel-2
S_clf_fn = base_path+'inputs-outputs/S2_classifier_all_sites.sav'
S_clf = pickle.load(open(S_clf_fn, 'rb'))
S_feature_cols_fn = base_path+'inputs-outputs/S2_feature_cols.pkl'
S_feature_cols = pickle.load(open(S_feature_cols_fn,'rb'))
# MODIS
M_clf_fn = base_path+'inputs-outputs/M_classifier_all_sites.sav'
M_clf = pickle.load(open(M_clf_fn, 'rb'))
M_feature_cols_fn = base_path+'inputs-outputs/M_feature_cols.pkl'
M_feature_cols = pickle.load(open(M_feature_cols_fn,'rb'))  

In [ ]:
# -----Classify snow
# Landsat
for date in L_col_xr.time.data:
    # create data frame to store pixel values
    L_df = pd.DataFrame(columns=L_band_names+['NDSI'])
    # extract pixel values
    for band in L_band_names:
        L_df[band] = L_col_xr.sel(time=date)[band].data.flatten()
    # find indices of rows without NaN
    I_real = np.where(pd.isnull(L_df).any(1)==False)[0]
    # drop rows with NaN
    L_df = L_df.dropna().reset_index(drop=True)
    # calculate NDSI (G-SWIR)/(G+SWIR)
    L_df['NDSI'] = (L_df['SR_B3'] -  L_df['SR_B6']) / (L_df['SR_B3'] +  L_df['SR_B6'])
    
    # classify SCA
    array_classified = L_clf.predict(L_df[L_feature_cols])
    
    # reshape from flat array to original shape
    im_classified = np.zeros((np.shape(L_col_xr['SR_B3'].sel(time=date).data)[0], np.shape(L_col_xr['SR_B3'].sel(time=date).data)[1]))
    im_classified[:] = np.nan
    im_classified[I_real] = array_classified
    
    # plot classified image
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))
    plt.rcParams.update({'font.size': 14, 'font.sans-serif': 'Arial'})
    # define x and y limits
    xmin, xmax = np.min(L_col_xr.sel(time=date).x.data)/1000, np.max(L_col_xr.sel(time=date).x.data)/1000
    ymin, ymax = np.min(L_col_xr.sel(time=date).y.data)/1000, np.max(L_col_xr.sel(time=date).y.data)/1000
    # RGB image
    ax1.imshow(np.dstack([L_col_xr['SR_B4'].sel(time=date), L_col_xr['SR_B3'].sel(time=date), L_col_xr['SR_B2'].sel(time=date)]), 
               extent=(xmin, xmax, ymin, ymax))
    ax1.set_xlabel("Easting [km]")
    ax1.set_ylabel("Northing [km]")
    # define colors for plotting
    color_snow = '#4eb3d3'
    color_ice = '#084081'
    color_rock = '#fdbb84'
    color_water = '#bdbdbd'
    # snow
    if any(im_classified.flatten()==1):
        ax2.imshow(np.where(im_classified == 1, 1, np.nan), cmap=matplotlib.colors.ListedColormap([color_snow, 'white']),
                    extent=(xmin, xmax, ymin, ymax))
        ax2.scatter(0, 0, color=color_snow, s=50, label='snow') # plot dummy point for legend
    if any(im_classified.flatten()==2):
        ax2.imshow(np.where(im_classified == 2, 4, np.nan), cmap=matplotlib.colors.ListedColormap([color_snow, 'white']),
                    extent=(xmin, xmax, ymin, ymax))
    # ice
    if any(im_classified.flatten()==3):
        ax2.imshow(np.where(im_classified == 3, 1, np.nan), cmap=matplotlib.colors.ListedColormap([color_ice, 'white']),
                    extent=(xmin, xmax, ymin, ymax))
        ax2.scatter(0, 0, color=color_ice, s=50, label='ice') # plot dummy point for legend
    # rock/debris
    if any(im_classified.flatten()==4):
        ax2.imshow(np.where(im_classified == 4, 1, np.nan), cmap=matplotlib.colors.ListedColormap([color_rock, 'white']),
                    extent=(xmin, xmax, ymin, ymax))
        ax2.scatter(0, 0, color=color_rock, s=50, label='rock') # plot dummy point for legend
    # water
    if any(im_classified.flatten()==5):
        ax2.imshow(np.where(im_classified == 5, 10, np.nan), cmap=matplotlib.colors.ListedColormap([color_water, 'white']),
                    extent=(xmin, xmax, ymin, ymax))
        ax2.scatter(0, 0, color=color_water, s=50, label='water') # plot
    ax2.set_xlim(xmin, xmax)
    ax2.set_ylim(ymin, ymax)
    plt.show()

In [ ]:
# create data frame to store pixel values
L_df = pd.DataFrame(columns=L_band_names+['NDSI'])
# extract pixel values
for band in L_band_names:
    L_df[band] = L_col_xr.sel(time=date)[band].data.flatten()
L_df